# Multi Agent Collaboration - News Writer
When you need more than a single agent to handle a complex task, you can create additional specialized agents to address different aspects of the process. However, managing these agents becomes technically challenging as tasks grow in complexity. As a developer using open source solutions, you may find yourself navigating the complexities of agent orchestration, session handling, memory management, and other technical aspects that require manual implementation.

With the fully managed multi-agent collaboration capability on Amazon Bedrock, specialized agents work within their domains of expertise, coordinated by a supervisor agent. The supervisor breaks down requests, delegates tasks, and consolidates outputs into a final response. For example, an investment advisory multi-agent system might include agents specialized in financial data analysis, research, forecasting, and investment recommendations. Similarly, a retail operations multi-agent system could handle demand forecasting, inventory allocation, supply chain coordination, and pricing optimization.

In this lab, we will build a multi-agent system where facts about a news event collected by a journalist are used to generate a news story. As shown in the diagram below, multiple agents will be responsible for tasks which are orchestrated by the supervisor agent:

<img src="../imgs/mac-news-generation.png" width="800">

The workflow shown in the diagram above is as follows:

1. Journalist uploads facts about a news event to an S3 bucket
2. The Interface Supervisor agent sends the facts to a Research Agent.
   1. The Research Agent is equipped with a Tool that triggers a Lambda function
   2. The Lambda Function calls a Bedrock Flow which does the following:
      1. Entity Extraction: These can be people, companies, products, etc.
      2. Gather background information: This uses the Bedrock Knowledge Base.
   3. The Lambda then returns the research to the Research Agent, which returns it to the Interface Supervisor
3. Once additional context has been provided by the Research Agent, the Interface Supervisor sends the research and the facts to the Article Generation Supervisor, which invokes the following agents:
   1. News Generation Agent: This writes the main news article based on the gathered information
   2. Article Reviewer Agent: This provides feedback to the News Generation Agent and iteratively improves the quality of teh generated article.
5. The remainder of the architecture is shown for completeness, and won't be part of this lab. Feel free to implement that if you have time at the end.

Please note that this is a simplified architecture to demonstrate multi-agent collaboration, a complete architecture would incorporate storing outputs at every stage for monitoring agents, and more opportunity for human-in-the-loop capability.

Let's get started!

## Amazon Bedrock

Amazon Bedrock Agents manages the collaboration, communication, and task delegation behind the scenes. By enabling agents to work together, you can achieve higher task success rates, accuracy, and enhanced productivity. In internal benchmark testing, multi-agent collaboration has shown marked improvements compared to single-agent systems for handling complex, multi-step tasks.

Highlights of multi-agent collaboration in Amazon Bedrock
A key challenge in building eﬀective multi-agent collaboration systems is managing the complexity and overhead of coordinating multiple specialized agents at scale. Amazon Bedrock simplifies the process of building, deploying, and orchestrating effective multi-agent collaboration systems while addressing efficiency challenges through several key features and optimizations:

- __Quick setup__ – Create, deploy, and manage AI agents working together in minutes without the need for complex coding.
- __Composability__ – Integrate your existing agents as subagents within a larger agent system, allowing them to seamlessly work together to tackle complex workflows.
- __Efficient inter-agent communication__ – The supervisor agent can interact with subagents using a consistent interface, supporting parallel communication for more efficient task completion.
- __Optimized collaboration modes__ – Choose between supervisor mode and supervisor with routing mode. With routing mode, the supervisor agent will route simple requests directly to specialized subagents, bypassing full orchestration. For complex queries or when no clear intention is detected, it automatically falls back to the full supervisor mode, where the supervisor agent analyzes, breaks down problems, and coordinates multiple subagents as needed.
 - __Integrated trace and debug console__ – Visualize and analyze multi-agent interactions behind the scenes using the integrated trace and debug console.

These features collectively improve coordination capabilities, communication speed, and overall effectiveness of the multi-agent collaboration framework in tackling complex, real-world problems.

## Create a News Research workflow using Amazon Bedrock Agents
In this section we declare global variables that will act as helpers during the entire notebook.
Here's a diagram that highlights the parts of the Research agent which we are going to build:

<img src="../imgs/mac-news-research.png" width="800">


In [ ]:
import boto3

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)


## Importing helper functions
On following section, we're adding bedrock_agent_helper.py on Python path, so the files can be recognized and their functionalities invoked.

In general, the helper functions handle common tasks including agent creation.


In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)

agents = AgentsForAmazonBedrock()

## Creating the Research Agent

Let's create the Research Agent that uses a Lambda tool to get research information from a Bedrock Flow.

The role of the agent is to extract, verify, and get information about entities present in the news facts input by a journalist.

We will provide the following instructions to our agent:
```
You are given a tool called action_group_research. This tool performs research. Your job is to return the outcome from the research to the caller.
Do not do any further modifications. Do not summarize the result.
```
We'll now Create a Bedrock Agent using the agent helper function

In [ ]:
agent_description = "An agent that gathers research on facts collected regarding a news event"
agent_instruction = """You are given a tool called action_group_research. This tool performs research. Your job is to return the outcome from the research to the caller.
Do not do any further modifications. Do not summarize the result."""
agent_foundation_model = [
    'us.amazon.nova-premier-v1:0'
]
# try: us.amazon.nova-pro-v1:0
# try: us.amazon.nova-premier-v1:0
# original, gets throttled: us.anthropic.claude-3-5-sonnet-20241022-v2:0

In [ ]:
import uuid

agent_suffix = str(uuid.uuid4())[:5]
agent_name = f"lab-7-research-agent-{agent_suffix}"

research_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False
)

research_agent

## Define Agent Tools
In the context of Bedrock agents, tools are organized as action groups. An action group defines actions that the agent can perform. For example, you could define an action group called `Get background research` that helps gather background research on entities that are provided to it. If the entities don't exist in the database, they will simply be ignored.

You create an action group by performing the following steps:

Define the parameters and information that the agent must elicit from the user for each action in the action group to be carried out.

Decide how the agent handles the parameters and information that it receives from the user and where it sends the information it elicits from the user.

To learn more about action groups, please refer to this [link](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html).

The function details consist of a list of parameters, defined by their name, data type (for a list of supported data types, see ParameterDetail), and whether they are required. The agent uses these configurations to determine what information it needs to elicit from the user. You can define the function detail in a JSON file with name, description, parameters, or provide a file in the OpenAPI compatible format.

To fulfill the task, you can define a Lambda function to program the business logic for an action group. After an Amazon Bedrock agent determines the API operation that it needs to invoke in an action group, it sends information from the API schema alongside relevant metadata as an input event to the Lambda function. To write your function, you must understand the following components of the Lambda function:

Input event – Contains relevant metadata and populated fields from the request body of the API operation or the function parameters for the action that the agent determines must be called.

Response – Contains relevant metadata and populated fields for the response body returned from the API operation or the function.

For more information please refer to this [link](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-lambda.html)

If a Lambda function is not feasible, another option is to choose to return control to the agent developer by sending the information in the InvokeAgent response. For more information please refer to this [link](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-returncontrol.html)

In this lab, we'll define a an action group with a Lambda function that simulates an API call. The API returns the research material on news entities, which are obtained from the knowledge bases that the agent can access.

In [ ]:
action_group_name = f"action_group_research"
action_group_descr = "Get background research about news entities"

function_defs = [
    {
        "name": "get_research_information",
        "description": "This function calls a flow to get research information.",
        "parameters": {},
        "requireConfirmation": "DISABLED"
    }
]

Create an action group and connect it to the lambda function that has already been written.

Feel free to go to the Lambda console to look at the code of the `lab6-lambda-CallFlowLambda` function

In [ ]:
%store -r call_flow_lambda_arn
%store -r call_flow_lambda_name

action_group_arn = agents.add_action_group_with_lambda(agent_name,
                                        call_flow_lambda_name, call_flow_lambda_arn,
                                        function_defs, action_group_name, action_group_descr)

## Testing the Agent
With all the components in place, let's test out our agent. Since the news facts are already in a file in S3, we'll ask a simple question to trigger the agent.

In [ ]:
%%time

response = agents.invoke(
        """get information without modifying it""",
        research_agent[0], enable_trace=True,
        trace_level="all"
)
print("====================")
print(response)
print("====================")
print("You are now ready to go to the next Lab!")